# Load stuff

In [1]:
import pandas as pd
import numpy as np
import re
import string
import time
from bs4 import BeautifulSoup
import spacy
nlp = spacy.load('en_core_web_sm')
from scipy.sparse import csr_matrix
import scipy.sparse as sps
import matplotlib.pyplot as plt
!pip install leidenalg # this is the Louvain version we use!!!!
import leidenalg
import igraph as ig
import pickle
from IPython.display import HTML
import matplotlib as mpl
import matplotlib.cm
!pip install umap-learn
import umap
import seaborn as sns

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.5 MB/s eta 0:00:00


In [2]:
!pip install bertopic
from bertopic import BERTopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [3]:
!pip install datamapplot
import datamapplot

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.0 MB/s eta 0:00:00
  Created wheel for Pyqtree: filename=Pyqtree-1.0.0-py3-none-any.whl size=5967 sha256=8f48ce7c87dff055a3986cd8fa12400aab5865d56c651b889411989f21285d8d
  Stored in directory: /root/.cache/pip/wheels/4c/87/93/c44242a244e4cea2cb9f5414ec450ababd7acac8b3377c3f9e
Succe

In [4]:
def logg(x):
    y = np.log(x)
    y[x==0] = 0
    return y

def nmi_fn(A): # A = Pwc
    aw = A.sum(axis=1).flatten() # word probability
    ac = A.sum(axis=0).flatten() # class probability
    Hc = np.multiply(ac,-logg(ac)).sum() # class entropy
    A2 = ((A/ac).T/aw).T
    A2.data = logg(A2.data)
    y = (A.multiply(A2)).sum()/Hc
    return y

def modularity_fn(A):
    y = A.trace()-(A.sum(axis=0)*A.sum(axis=1)).item()
    return y

def ncut_fn(A):
    y = ((A.sum(axis=0)-A.diagonal())/A.sum(axis=0)).mean()
    return y

def pagerank_fn(M,q,c=.85,it=60):
    r = q.copy() # ranking matrix, initialized to q (copy)
    for k in range(it): # slow cycle
      r = c*M.dot(r) + (1-c)*q
    return r

def _infomap_fn(v):
    y = -(v.data*logg(v.data/v.sum())).sum()
    return y

def infomap_rank_fn(Pdd):
    # transition matrix
    pd = Pdd.sum(axis=0).flatten()
    M = sps.csr_matrix(Pdd/pd)
    # pagerank vector - faster than r = pagerank_fn(M,q)
    G = ig.Graph.Adjacency((M > 0).toarray().tolist())
    G.es['weight'] = np.array(M[M.nonzero()])[0]
    r = G.pagerank(weights='weight')
    r = (sps.csr_matrix(np.array(r))).T
    return r

def infomap_fn(C,Pdd,r):
    pd = Pdd.sum(axis=0).flatten()
    M = Pdd/pd # transition matrix
    # extract vectors
    z = (C.T).dot(sps.diags(r.toarray().flatten()))
    q = sps.csr_matrix((1,z.shape[0]))
    c = .85
    for i in range(z.shape[0]):
      tmp = ((C[:,i].transpose()).dot(M)).dot(z[i].transpose())
      q[0,i] = (1-(1-c)*C[:,i].sum()/M.shape[0])*z[i].sum()-c*tmp[0,0]
    # extract statistics
    y = _infomap_fn(q)
    for i in range(z.shape[0]):
      y += _infomap_fn(sps.hstack([z[i],sps.csr_matrix([[q[0,i]]])]))
    # normalize
    y = (y/_infomap_fn(pd))-1
    return y

def _hlstr(string: str , color: str = 'white'):
    # return f"<mark style=background-color:{color}>{string} </mark>"
    return f"<span style=\"color:{color}\">{string} </span>"

def _colorize(attrs: np.ndarray, cmap: str = 'PiYG'):
    norm = mpl.colors.Normalize(vmin=-1, vmax=1)
    cmap = matplotlib.colormaps.get_cmap(cmap)
    return list(map(lambda x: mpl.colors.rgb2hex(cmap(norm(x))), attrs))

def _display_html(tokens, attrs, co=1):
    return HTML("".join(list(map(_hlstr, tokens, _colorize(co*attrs)))))

def get_words_html(vec,wor,N):
    ind = np.argsort(-vec)
    wor = wor[ind[:N]]
    vec = vec[ind[:N]]
    vec = vec/vec[0]
    for j in range(len(wor)):
        wor[j] = str(nlp(str(wor[j]))[0].lemma_)
    return _display_html(wor,vec).data

In [5]:
class TopicDetection:
    # builds an occurrence matrix for words
    def __init__(self, ct_class=[], n_min=2, n_max=1e10):
      if ct_class:
        # capture execution time
        tic = time.time()
        # collection of words
        self.words = np.unique([item for sublist in ct_class.pos_clean for item in sublist])
        # words dictionary
        words_dict = dict(zip(self.words,range(len(self.words))))
        # occurrence matrix for words
        self.Mwd = csr_matrix((len(self.words), len(ct_class.text)), dtype = np.int8).toarray()
        for i in range(len(ct_class.text)):
            for j in ct_class.pos_clean[i]:
                self.Mwd[words_dict[j],i] += 1
        # documents list
        self.documents = np.array(range(len(ct_class.text)))
        # remove words that are used once or twice
        select = ((np.sum(self.Mwd,axis=1)>=n_min)&(np.sum(self.Mwd,axis=1)<n_max))
        print('removing words...')
        with np.printoptions(threshold=np.inf):
          print(self.words[(np.sum(self.Mwd,axis=1)>=n_max)])
        self.Mwd = self.Mwd[select,:]
        self.words = self.words[select]
        # remove documents that do not contain words
        select = (np.sum(self.Mwd,axis=0)>0)
        self.Mwd = csr_matrix(self.Mwd[:,select])
        self.documents = self.documents[select]
        # capture execution time
        print(f'Occurrence matrix: execution time {time.time()-tic} [s]')

    # plots words occurrencies
    def plot_word_occurrence(self):
        plt.figure(figsize=(4, 3))
        plt.semilogy(-np.sort(-np.asarray(np.sum(td1.Mwd,axis=1)).reshape(-1)))
        plt.grid(True)
        plt.xlabel('word id')
        plt.ylabel('# of occurrences')
        plt.title("words occurrencies")

    # plots degree distributions
    def plot_degree_distributions(self,quale='words'):
        if quale=='words':
            dw = np.squeeze(np.asarray(self.Mwd.sum(axis=1))) # words degrees
        else:
            dw = np.squeeze(np.asarray(self.Mwd.sum(axis=0))) # documents degrees
        # degree distribution
        k = np.unique(dw)
        pk = np.histogram(dw,np.append(k,k[-1]+1))[0]
        pk = pk/pk.sum()
        Pk = 1-np.cumsum(pk)
        # plot
        fig = plt.figure(figsize=(4, 3))
        plt.loglog(k,pk,'o')
        if quale=='words':
            plt.title("degree distribution for words")
        else:
            plt.title("degree distribution for documents")
        plt.xlabel("k")
        plt.ylabel("pk")
        plt.grid(True)
        plt.show()

    # build probability matrices
    def probability_matrices(self, equalik = True, tform = False):
        # capture execution time
        tic = time.time()
        if equalik: # documents equally likely
            self.Pwd = self.Mwd/self.Mwd.sum(axis=0).flatten()/self.Mwd.shape[1]
        else: # documents proportional to their length
            self.Pwd = self.Mwd/self.Mwd.sum()
        # TF-IDF format
        if (tform):
            iw = -logg(np.sum(self.Mwd>0,axis=1).flatten()/self.Mwd.shape[1])
            self.Pwd = sps.diags(np.array(iw)[0]).dot(self.Pwd) # TF-IDF form
            self.Pwd = self.Pwd/self.Pwd.sum() # normalize, treat it as Pwd
        # words and document matrices
        self.pd = self.Pwd.sum(axis=0).flatten()
        self.Pww = (self.Pwd/self.pd).dot(self.Pwd.T)
        self.pw = self.Pwd.sum(axis=1).flatten()
        self.Pdd = (self.Pwd.T/self.pw).dot(self.Pwd)
        # # joint words and document matrix - documents first
        # self.Paa = sps.hstack((sps.csr_matrix((self.Pwd.shape[1],self.Pwd.shape[1])),self.Pwd.T))
        # self.Paa = sps.vstack((self.Paa,sps.hstack((self.Pwd,sps.csr_matrix((self.Pwd.shape[0],self.Pwd.shape[0]))))))
        # self.Paa = self.Paa/2.0
        # ranking vector for Infomap
        self.rd = infomap_rank_fn(self.Pdd)
        # capture execution time
        print(f'Probability matrices: execution time {time.time()-tic} [s]')

    # run topic detection using Louvain
    def topic_detection_louvain(self):
        # capture execution time
        tic = time.time()

        # build a graph based on Pdd as adjacency matrix
        A = sps.csr_matrix(self.Pdd)
        G = ig.Graph.Adjacency((A > 0).toarray().tolist())
        G.es['weight'] = np.array(A[A.nonzero()])[0]
        # run Louvain on the graph to get a partition
        self.topics = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition,
                                        weights='weight')
        # map the partition into a community assignment matrix C
        self.C = self._partition_to_C()
        self.topics = [np.argmax(i) for i in self.C] # turn C into a vector!
        # updates topic matrices
        self.Pwc = self.Pwd.dot(self.C) # joint word + class probability
        self.Pcc = ((self.C.T).dot(self.Pdd)).dot(self.C) # joint class + class probability
        self.pc = self.Pcc.sum(axis=0)

        # capture execution time
        ext = time.time()-tic
        print(f'Louvain topic detection: execution time {ext} [s]')
        return ext

    # function to map the partition into a community assignment matrix C
    # where rows represent documents, and columns represent topics
    def _partition_to_C(self):
      C = sps.csr_matrix((self.Pdd.shape[0],len(self.topics)))
      for i in range(len(self.topics)):
        C[np.array(self.topics[i]),i] = 1
      return C

    # display clustering metrics
    def clustering_metrics(self, ext=np.inf):
        NMI = nmi_fn(self.Pwc)
        Q = modularity_fn(self.Pcc)
        Ncut = ncut_fn(self.Pcc)
        Infomap = infomap_fn(self.C,self.Pdd,self.rd)
        if (self.pc.shape[1]==1):
            com = 0
        else:
            com = _infomap_fn(self.pc)/np.log(self.pc.shape[1])
        if ext==np.inf:
          self.stats = {'topics': self.C.shape[1], 'com': com,
                        'NMI': NMI, 'Q': Q, 'Ncut': Ncut, 'Infomap': Infomap}
        else:
          self.stats = {'topics': self.C.shape[1], 'com': com, 'time': ext,
                        'NMI': NMI, 'Q': Q, 'Ncut': Ncut, 'Infomap': Infomap}

    # display clusters
    def display_clusters(self):
        # ordered cluster matrix
        pos = np.argsort(np.array([np.argmax(i) for i in self.C]))
        C = sps.csr_matrix(self.C.copy()).astype(np.float32)
        C = C[pos,]
        M = C*(C.T) # binary matrix with ones inside the communities
        # plot
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(6, 3))
        ax[0].plot(np.array(self.C.sum(axis=0))[0]/self.C.sum(),'o')
        ax[0].set_title('clusters dimensions')
        ax[0].grid(True)
        ax[1].imshow(M.toarray(), cmap='viridis')
        ax[1].set_xticklabels([])
        ax[1].set_yticklabels([])
        ax[1].set_title('clusters patterns')

    # display keywords
    def display_keywords(self, text, N=15):
      # build HTML array according to the weights in Pwc
      x = []
      for i in range(self.Pwc.shape[1]):
        x.append([get_words_html(self.Pwc.T[i].toarray()[0],self.words,N)])
      # build HTML array according to TF-IDF weigts in BERTopic
      topic_model = BERTopic() # prepare a BERTopic instance
      topic_model.fit(text)
      topic_model.update_topics(text, topics=self.topics, top_n_words=N)
      bx = [] #
      for k in range(self.C.shape[1]):
        wor = np.array([i for (i,j) in topic_model.get_topic(k)])
        vec = np.array([j for (i,j) in topic_model.get_topic(k)])
        bx.append([get_words_html(vec,wor,N)])
      # join them together and add titles
      c = np.concatenate(([[f'{i}'] for i in range(self.C.shape[1])],
                          [[f'{int(i)}'] for i in np.array(self.C.sum(axis=0))[0]],
                          np.array(x),np.array(bx)), axis=1)
      c = np.concatenate(([['Topic','#docs','Probability matrices keywords',
                                    'BERTopic keywords']],c))
      # map into HTML table and print
      out = '<table><col><col><col width=\"250\"><col width=\"250\">'
      out = out + ''
      for sublist in c:
          out = out + ' <tr><td style=\"padding:1px 8px;\" valign=\"top\">'
          out = out + ' </td><td style=\"padding:1px 8px;\" valign=\"top\">'.join(sublist)
          out = out + ' </td></tr>'
      out = out + ' </table>'
      display(HTML(out))

    # export data
    def pickle_export(self, filename):
        out_data = {
            'words': self.words,
            'documents': self.documents,
            'Mwd': self.Mwd,
            'Pwd': self.Pwd,
            'Pww': self.Pww,
            'Pdd': self.Pdd,
            'pd': self.pd,
            'pw': self.pw,
            'rd': self.rd,
            'Pwc': self.Pwc,
            'Pcc': self.Pcc,
            'pc': self.pc,
            'Pwc': self.Pwc,
            'Pcc': self.Pcc,
            'pc': self.pc,
            'C': self.C,
            'topics': self.topics,
            'stats': self.stats
        }
        with open(filename, 'wb') as f:
            pickle.dump(out_data, f)

    # import data
    def pickle_import(self, filename):
        with open(filename, 'rb') as f:
            in_data = pickle.load(f)
        self.words = in_data['words']
        self.documents = in_data['documents']
        self.Mwd = in_data['Mwd']
        self.Pwd = in_data['Pwd']
        self.Pww = in_data['Pww']
        self.Pdd = in_data['Pdd']
        self.pd = in_data['pd']
        self.pw = in_data['pw']
        self.rd = in_data['rd']
        self.Pwc = in_data['Pwc']
        self.Pcc = in_data['Pcc']
        self.pc = in_data['pc']
        self.Pwc = in_data['Pwc']
        self.Pcc = in_data['Pcc']
        self.pc = in_data['pc']
        self.C = in_data['C']
        self.topics = in_data['topics']
        self.stats = in_data['stats']

# Load data

In [22]:
# load dataframe
df = pd.read_csv("CC_comments.csv")
df.head()

,Unnamed: 0,0
0,0,love content have suggestion far research topic
1,1,climate change use TOO MUCH
2,2,last time reach ocean
3,3,need massive desalination plant major sw city ...
4,4,mind blow


In [23]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Definire un CountVectorizer che rimuove le stopwords
vectorizer_model = CountVectorizer(stop_words="english")  # Rimuove stopwords in inglese

In [8]:
"""from sklearn.feature_extraction.text import CountVectorizer
custom_stopwords = ["you", "all", "one", "just", "thing", "go", "be", "it", "to", "that", "the", "is", "and",
                    "of", "its", "in", "we", "my", "this", "so", "are", "will", "for", "have", "not", "she", "their", "on",
                    "what", "they", "us", "but", "do", "im", "no", "had", "with"]
vectorizer_model = CountVectorizer(stop_words=custom_stopwords)"""

'from sklearn.feature_extraction.text import CountVectorizer\ncustom_stopwords = ["you", "all", "one", "just", "thing", "go", "be", "it", "to", "that", "the", "is", "and",\n                    "of", "its", "in", "we", "my", "this", "so", "are", "will", "for", "have", "not", "she", "their", "on",\n                    "what", "they", "us", "but", "do", "im", "no", "had", "with"]\nvectorizer_model = CountVectorizer(stop_words=custom_stopwords)'

# BERTopic

In [25]:
topic_model = BERTopic(nr_topics="auto", vectorizer_model=vectorizer_model)
docs = df["0"].astype(str)
topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1596,-1_people_year_say_live,"[people, year, say, live, just, think, thank, ...","[try thank good thing, really think let happen..."
1,0,2186,0_climate_change_water_say,"[climate, change, water, say, just, time, know...",[say go happen then again then again then agai...
2,1,87,1_lawn_grass_flower_snake,"[lawn, grass, flower, snake, yard, insect, bug...","[show lawn, Flower lawn 🔛 🔝, dad mow lawn alwa..."
3,2,72,2_tree_cut_plant_forest,"[tree, cut, plant, forest, wood, building, hou...","[plant tree, even cut tree, so tree 🙄]"
4,3,37,3_house_home_build_rebuild,"[house, home, build, rebuild, new, baxter, dis...","[company still build house want so bad, house,..."
5,4,32,4_nan___,"[nan, , , , , , , , , ]","[nan, nan, nan]"
6,5,30,5_eat_meal_single_health,"[eat, meal, single, health, day, good, crumb, ...",[eat single meal year feel very good health ve...
7,6,28,6_plastic_steak_micro_microplastic,"[plastic, steak, micro, microplastic, straw, m...",[currently work micro plastic soil lab soil sa...
8,7,24,7_base_red_unbelievably_template,"[base, red, unbelievably, template, redpill, p...","[base, base, base]"
9,8,21,8_boost_fyp_favorite_color,"[boost, fyp, favorite, color, emoji, puma, pac...","[Boost, boost boost, boost]"


In [26]:
topic_model.visualize_barchart(top_n_topics=10, n_words=7)

In [27]:
# Definizione dei nomi personalizzati per i topi
custom_labels = {
    0: "Poor People, Heart Cries",
    1: "Faith",
    2: "Enviornment",
    3: "Window",
    4: "Google, AI, nuclear",
    5: "Manifest, Trumpets",
    6: "Bathroom"
}

# Assegna i nuovi nomi ai topic
topic_model.set_topic_labels(custom_labels)

# Verifica le nuove etichette assegnate
topic_model.get_topic_info()

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,1596,-1_people_year_say_live,-1_people_year_say_live,"[people, year, say, live, just, think, thank, ...","[try thank good thing, really think let happen..."
1,0,2186,0_climate_change_water_say,"Poor People, Heart Cries","[climate, change, water, say, just, time, know...",[say go happen then again then again then agai...
2,1,87,1_lawn_grass_flower_snake,Faith,"[lawn, grass, flower, snake, yard, insect, bug...","[show lawn, Flower lawn 🔛 🔝, dad mow lawn alwa..."
3,2,72,2_tree_cut_plant_forest,Enviornment,"[tree, cut, plant, forest, wood, building, hou...","[plant tree, even cut tree, so tree 🙄]"
4,3,37,3_house_home_build_rebuild,Window,"[house, home, build, rebuild, new, baxter, dis...","[company still build house want so bad, house,..."
5,4,32,4_nan___,"Google, AI, nuclear","[nan, , , , , , , , , ]","[nan, nan, nan]"
6,5,30,5_eat_meal_single_health,"Manifest, Trumpets","[eat, meal, single, health, day, good, crumb, ...",[eat single meal year feel very good health ve...
7,6,28,6_plastic_steak_micro_microplastic,Bathroom,"[plastic, steak, micro, microplastic, straw, m...",[currently work micro plastic soil lab soil sa...
8,7,24,7_base_red_unbelievably_template,7_base_red_unbelievably_template,"[base, red, unbelievably, template, redpill, p...","[base, base, base]"
9,8,21,8_boost_fyp_favorite_color,8_boost_fyp_favorite_color,"[boost, fyp, favorite, color, emoji, puma, pac...","[Boost, boost boost, boost]"


In [12]:
topic_model.visualize_topics()

In [13]:
topic_model.visualize_heatmap()

In [29]:
topic_model.visualize_documents(df['0'].astype(str))

# Reducing outliers

In [30]:
new_topics = topic_model.reduce_outliers(docs, topics)
topic_model.update_topics(docs, topics=new_topics)
topic_model.get_topic_info()

2025-02-10 10:55:05,050 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,102,-1_over_outstanding_biggering_tay,-1_people_year_say_live,"[over, outstanding, biggering, tay, nuke, tour...","[try thank good thing, really think let happen..."
1,0,3001,0_so_have_say_do,"Poor People, Heart Cries","[so, have, say, do, climate, change, go, year,...",[say go happen then again then again then agai...
2,1,126,1_lawn_flower_grass_yard,Faith,"[lawn, flower, grass, yard, garden, snake, ins...","[show lawn, Flower lawn 🔛 🔝, dad mow lawn alwa..."
3,2,117,2_tree_cut_plant_forest,Enviornment,"[tree, cut, plant, forest, wood, fake, park, h...","[plant tree, even cut tree, so tree 🙄]"
4,3,90,3_house_home_family_new,Window,"[house, home, family, new, live, move, rebuild...","[company still build house want so bad, house,..."
5,4,33,4_nan___,"Google, AI, nuclear","[nan, , , , , , , , , ]","[nan, nan, nan]"
6,5,62,5_eat_meal_day_single,"Manifest, Trumpets","[eat, meal, day, single, health, good, only, v...",[eat single meal year feel very good health ve...
7,6,48,6_plastic_steak_micro_microplastic,Bathroom,"[plastic, steak, micro, microplastic, meat, an...",[currently work micro plastic soil lab soil sa...
8,7,34,7_base_red_extremely_twice,7_base_red_unbelievably_template,"[base, red, extremely, twice, mars, correct, j...","[base, base, base]"
9,8,29,8_boost_favorite_fyp_leo,8_boost_fyp_favorite_color,"[boost, favorite, fyp, leo, games, fit, speed,...","[Boost, boost boost, boost]"


In [31]:
topic_model.visualize_documents(docs)